In [1]:
import cloudscraper

In [2]:
url = 'https://www.zonaprop.com.ar/departamentos-alquiler.html'


In [3]:
scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
# Or: scraper = cloudscraper.CloudScraper()  # CloudScraper inherits from requests.Session
web_page = scraper.get(url)
text = web_page.text

In [4]:
web_page

<Response [200]>

In [5]:
from bs4 import BeautifulSoup


In [6]:
soup = BeautifulSoup(text, 'lxml')

In [26]:
soup.find_all('h1')[0].text

# get the number with regex and convert into int

import re

match = re.findall(r'\d+\.?\d+', soup.find_all('h1')[0].text)[0]

#replace the dot with nothing

match = match.replace('.', '')

#convert into int

int(match)


14993

In [8]:
import re

In [9]:
# Find h1 with text with regex matching "14.810 Departamentos"
apartment_quantity = soup.find_all('h1', text=re.compile(r'\d+ Departamentos'))[0].text
apartment_quantity

'14.993 Departamentos en alquiler en Argentina'

In [10]:
estate_posts = soup.find_all('div', attrs = {'data-posting-type' : True})

In [11]:
len(estate_posts)

20

In [12]:
one_estate = estate_posts[0]

In [13]:
# find div with anything data-qa atributte
data_qa = one_estate.find_all('div', attrs={'data-qa': True})

In [14]:
len(data_qa)

6

In [72]:
def parse_money_value(text):
    money_value = re.findall(r'\d+\.?\d+', text)[0]
    money_value = money_value.replace('.', '')
    money_value = int(money_value)
    money_type = re.findall(r'(USD)|(ARS)|(\$)', text)[0]
    money_type = [x for x in money_type if x != ''][0]
    return money_value, money_type

def parse_text(text):
    text = text.replace('\n', '')
    text = text.replace('\t', '')
    text = text.strip()
    return text

def parse_features(text):

    features_matches = re.compile(r'(\d+\.?\d*)\s(\w+)').findall(text)

    feature_unit_dict = {'m²': 'square_meters_area', 'amb': 'rooms', 'dorm': 'bedrooms', 'baños': 'bathrooms'}
    features_appearance = {'square_meters_area': 0, 'rooms': 0, 'bedrooms': 0, 'bathrooms': 0}

    features = {}

    for feature in features_matches:
        feature_unit = f'{feature_unit_dict[feature[1]]}_{features_appearance[feature_unit_dict[feature[1]]]}'
        features_appearance[feature_unit_dict[feature[1]]] += 1
        features[feature_unit] = feature[0]
    return features

In [61]:
string = '115 m²  110 m²  3 amb.  2 dorm.  2 baños'

In [71]:
# make a regex that matches the number and unit of all items on the string

features = re.compile(r'(\d+\.?\d*)\s(\w+)').findall(string)

feature_unit_dict = {'m²': 'square_meters_area', 'amb': 'rooms', 'dorm': 'bedrooms', 'baños': 'bathrooms'}

for feature in features:
    print(feature[0], feature_unit_dict[feature[1]])

115 square_meters
110 square_meters
3 rooms
2 bedrooms
2 bathrooms


In [69]:
features

[('115', 'm²'), ('110', 'm²'), ('3', 'amb'), ('2', 'dorm'), ('2', 'baños')]

In [73]:
for data in data_qa:
    label = data['data-qa']
    text = None
    if label in ['POSTING_CARD_PRICE', 'expensas']:
        text = parse_money_value(data.get_text())
    elif label in ['POSTING_CARD_LOCATION', 'POSTING_CARD_DESCRIPTION']:
        text = parse_text(data.get_text())
    elif label in ['POSTING_CARD_FEATURES']:
        text = parse_features(data.get_text())
    else:
        text = data.get_text()

    print(label, text)

POSTING_CARD_GALLERY 
POSTING_CARD_PRICE (2000, 'USD')
expensas (34640, '$')
POSTING_CARD_LOCATION Palermo Chico, Palermo
POSTING_CARD_FEATURES {'square_meters_0': '115', 'square_meters_1': '110', 'rooms_0': '3', 'bedrooms_0': '2', 'bathrooms_0': '2'}
POSTING_CARD_DESCRIPTION Departamento en Alquiler en edificio de categoría. Sin muebles. Living-comedor. Toilette de recepción. Un dormitorio principal con balcón. Escritorio/ Playroom o segundo dormitorio. Un baño completo. Amplia cocina reciclada y equipada con heladera, lavavajillas, cocina a gas. Lavadero (lavasecarropas) y dependencia de servicio con baño. Artefactos eléctricos de iluminación instalados y cortinas. Cerramientos aislantes nuevos. No tiene cochera, pero posibilidad de alquiler en edificio de enfrente. Seguridad 24hs. Pileta climatizada. Aberturas de pvc y doble vidrio. Hay tres equipos de aire acondicionado: en el living-comedor y en cada dormitorio. El departamento está disponible a partir de principios de Agosto. Par

In [19]:
i = 1
print(data_qa[i]['data-qa'])
print(data_qa[i].text)


POSTING_CARD_PRICE
USD 2.000
